### BeautifulSoup Modülü Kullanarak Ebay'den Veri Çekme

##### 1. Kullanılan Kütüphaneler
    1.BeautifulSoup : HTML veya XML dosyalarını işlemek için oluşturulmuş güçlü ve hızlı bir kütüphanedir.
    2.requests : HTTP protokolü ile ilgili işlemlerinizi kolayca yapmanızı sağlayan kütüphanedir.
    3.re : Regular Expressions (Regex), Türkçesi ise Düzenli İfadeler olarak geçer. Regex bir string ifadenin (metin) belirli kurallara uyumluluğunu kontrol etmeye ve düzenlemeye yarar.
    4.ast : AST(for Abstract Syntax Tree) Ayrıştırma amacı, bilgilerin metinsel bir temsilini veri yapıları temsiline veya AST'ye (Özet Sözdizimi Ağacı için) çevirmektir.
    5.csv
    6.time

##### 2. Converter Api
    Ebay.com USA sitesi olduğundan farklı para birimleri Amerikan Doları(USD) olarak çeviriliyor.      
    Avrupa piyasası incelendiği için tüm para birimlerini Euro(EUR) olarak çevirmek için ücretsiz api kullanıldı.
    https://api.exchangeratesapi.io/latest?base=USD

##### 3. Ebay'den Veri Çekerken Engellenmemek 
    Session objesi aynı anda yapılan birkaç requests'in tek bir oturumda gerçekleşmesini sağlıyor.
    https://www.geeksforgeeks.org/session-objects-python-requests/

##### 4. response.text ve response.content
    content: Bu özellik, yanıt içeriğinin ham baytını döndürür.
    text: Text özelliği, içeriği normal bir UTF-8 kodlu Python dizesi olarak döndürür. 
    Content kullanıldığında lxml kütüphanesi bazı html elementleri yorumlayamıyor bu yüzden "AttributeError: 'NoneType' object has no attribute 'text' " hatası alınıyor.
    Bu nedenle text kullanıldı.

##### 5. for i in range(1, 200)
    Ebay sitesinde 200 sayfadan fazlasını görüntülemek istediğimizde sitede  "We're unable to show you more than 10,000 results. Please refine your search to narrow your results." yazısıyla karşılaşıyoruz. Bu nedenle limitimizi 200 olarak alındı.

##### 6. Aynı Sayfadan Tekrar Veri Çekmemek 
    same_page adında boş bir liste oluşturulup sayfadan alınan ilk ürün linki bu listeye eklendi. Sayfadaki ürün linki
     O'dan büyük ve alınan ilk ürün o sayfada 1'e eşit ise ürünün linkine gidildi değil ise break komutuyla işlem durduruldu. Bu adımla aynı üründen veri çekme engellendi.

##### 7. None_check Fonksiyonu
    Seçilen ürünün verisinin var olup olmadığını kontrol edip tek seferde strip'leyerek daha okunaklı ve hızlı performans elde etmek için oluşturuldu.


##### 8. Para Birimi
    Farklı para biriminde satılan ürünlerin  USD dolar değeri  'convbinPrice'  alınarak seçildi ve kullanılan apideki para birimiyle çarpılarak veri çekilen andaki EUR değeri hesaplandı.

##### 9. Kargo Ücreti
    Ürünlerin kargo ücretinde "FREE", "May not ship to Turkey", "Doesn't Ship to Turkey","Standart Shipping", "USD" veya farklı para birimi ile karşılaşılıyor. Her durumda değişen attributeleri  if/else içinde kullanarak alındı.

##### 10. Farklı Renkteki Ürünlerin Değişen Fİyatları
    Ebay sitesinde farklı renkleri çekmeye çalıştığımızda html dilinde  olmadığı anlaşıldı. Bilgiler Javascript işlenmiş  json formatında bulundu. Json formatındaki istenilen veri regex ile alındı.
    https://jsonformatter.curiousconcept.com/
    https://regex101.com/

##### 11.Veri Çekme Zamanını Kısaltma
    Ebayde  sayfada gösterilen ürün sayısı 50 veya daha düşük olunca sayfa sayısı artıyor.Bu durum her sayfaya requests atıp bekleme süresini oldukça uzatıyor. Bundan kaçınmak için sayfa başına gösterilen ürün sayısı maksimuma çekildi bu site için 200 oldu. Sayfa sayısı azaldığı için atılacak requests sayısı azaldı hem bekleme süresi azaldı hem de  kodun performansı artmış oldu.

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re
import ast

class ebay(object):
    def __init__(self, url):
        self.same_page = list()
        self.usd=float(1*requests.get('https://api.exchangeratesapi.io/latest?base=USD').json()['rates']['EUR'])
        self.req = requests.session()
        self.get_page(url)

    def get_page(self, url):
        for i in range(1, 200):
            response = self.req.get(url + "&_pgn=" + str(i))
            time.sleep(2.4)
            print("Printed after 2.4 seconds.")
            if response.ok:
                soup = BeautifulSoup(response.text, 'lxml')
                hash_finaly=soup.find_all('a', class_='s-item__link', href=True)[0].text
                self.same_page.append(hash_finaly)
                if len(soup.find_all('a', class_='s-item__link', href=True)) > 0 and self.same_page.count(hash_finaly)==1:
                    print("Page {} is parsing.".format(i))
                    self.get_product_detail(soup)
                else:
                    print("All data was scraped")
                    break
            else:
                print('Server responded:', response.status_code)
                
    def none_check(self, var):
        if var is not None:
            return var.getText(strip=True)
        else:
            return "Null"
        
    def get_product_detail(self, source):
        if True:
            for i in source.find_all('a', class_='s-item__link', href=True):
                self.detail = self.req.get(i['href'])
                soup1 = BeautifulSoup(self.detail.text, 'lxml')
                print("Retrieving product data ",soup1.title.text.split("|")[0])
                
                title=soup1.title.text.split('|')[0].strip()
                
                location = self.none_check(soup1.find('span', attrs={'itemprop': 'availableAtOrFrom'}))
                
                sellername = self.none_check(soup1.find('span', attrs={'class': 'mbg-nw'}))
                
                condition = self.none_check(soup1.find('div', attrs={'itemprop': "itemCondition"}))
                
                brand=self.none_check(soup1.find('h2',attrs={'itemprop':'brand'}))
                
                try:
                    if soup1.find('span', attrs={'id':'convbidPrice'}):
                        continue
                    else:
                        if soup1.find('span', attrs={'id':'convbinPrice'}):
                            price=soup1.find('span', attrs={'id':'convbinPrice'}).text
                        else:
                            price = soup1.find("span", attrs={'itemprop': 'price'}).text
                    price='EUR '+'{:.2f}'.format(float(str(price.split("$")[1]).split('(')[0].replace(',',''))*self.usd)
                except AttributeError as error:
                    price = "Null"
                    
                if soup1.find('span',attrs={'id':'convetedPriceId'}):
                    shipping_cost=soup1.find('span',attrs={'id':'convetedPriceId'}).text
                else:
                    if soup1.find('span',attrs={'id':'fshippingCost'}):
                        if re.match(r"\$([0-9\.\,]*)",soup1.find('span',attrs={'id':'fshippingCost'}).text):
                            shipping_cost=soup1.find('span',attrs={'id':'fshippingCost'}).text
                        else:
                            shipping_cost=soup1.find('span',attrs={'id':'fshippingCost'}).text.strip()
                    else:
                        shipping_cost=''.join(soup1.find('span', attrs={ 'id': 'shSummary' }).find('span',attrs={'class':re.compile('[a-zA-Z]')}).text.strip())
                        
                if re.findall(r"\$([0-9\.\,]*)",shipping_cost):
                    shipping_cost='EUR '+str(int(float(re.findall(r"\$([0-9\.\,]*)",shipping_cost)[0].replace(',',''))*self.usd))

                data = {
                    'Title': title,
                    'Brand':brand,
                    'Price': price,
                    'Location': location,
                    'Shipping cost': shipping_cost,
                    'Sellername': sellername,
                    'Condition': condition
                }
                
                if soup1.find('select', attrs={'name': re.compile('Color*')}):
                    self.ebay_parser(self.detail.text,data)
                else:
                    self.csv_write(data)
                
    def ebay_parser(self, raw_data,product_data):
        try:
            regex = re.compile("\$rwidgets\((.*.)\);new\s?\(raptor\.require")
            data = regex.search(raw_data).group(1)
            valid_data = data.replace("true", "True").replace("false", "False").replace("null", "None")
            parsed = ast.literal_eval(valid_data)
            for k,i in enumerate(parsed):
                if i[0]=="com.ebay.raptor.vi.msku.ItemVariations":
                    for c in parsed[k][2]['itmVarModel']['menuItemMap']:
                        product_data['Color']=parsed[k][2]['itmVarModel']['menuItemMap'][c]['valueName']
                        product_data['Price']='EUR '+'{:.2f}'.format(float(str(str(parsed[k][2]['itmVarModel']['itemVariationsMap'][str(parsed[k][2]['itmVarModel']['menuItemMap'][c]['matchingVariationIds'][0])]['convertedPrice']).split("$")[1]).split('(')[0].replace(',',''))*self.usd)
                        self.csv_write(product_data)
        except Exception as error:
            pass
    
    def csv_write(self, data):
        local_data = list()
        csvfile = open('output01.csv', 'a', encoding='utf-8')
        writer = csv.writer(csvfile, delimiter=',')
        for i in data.values():
            local_data.append(i)
        writer.writerow(local_data)
        
ebay('https://www.ebay.com/sch/i.html?_nkw=robotic+vacuum+cleaner&LH_PrefLoc=5&_ipg=200')

Printed after 2.4 seconds.
Page 1 is parsing.
Retrieving product data  1800PA Multifunctional Smart Floor Cleaner Vacuum Robot 3-In-1 Auto Rechargeable  
Retrieving product data  360 CLEANX Laser navigated Robotic Vacuum Cleaner with 2000pa and AI integration  
Retrieving product data  Automatic Smart Robot Vacuum Cleaner Small Vacuum Cleaners Sweeping Robot Floor   
Retrieving product data  Cecotec Conga Series 3090 Robot Vacuum Cleaner 4 on 1 with Mapping and App.  
Retrieving product data  USB Automatic Robot Vacuum Cleaner Floor Dust Collector Smart Robot  
Retrieving product data  EUFY ROBOVAC 11 MODEL T2102 HIGH SUCTION SELF-DOCKING  ROBOTIC VACUUM CLEANER   
Retrieving product data  Samsung robot floor vacuum cleaner VR10M701BUW/SB  
Retrieving product data  H.Koenig Robot Vacuum Cleaner Bagless Vacuum Robot Cleaning Robot Charger  
Retrieving product data  EU UK Plug Global Version Midea MR01 02 Robot Vacuum Cleaner Suction Mopping  
Retrieving product data  Charging base for R

Retrieving product data  Cecotec Robot Vacuum Cleaner Conga Series 1590 Active 1400Pa Alexa&google App W  
Retrieving product data  Neato Robotics D701 Connected Vacuum Cleaner Robot Station of Charging, Wi-Fi &  
Retrieving product data  Robotic Vacuum Cleaner, Smart USB Robot Vacuum Cleaner High Suction.  
Retrieving product data  Cecotec 899 Robot Vacuum Cleaner 2 on 1 5 Modes of Cleaning, Navigation Smart  
Retrieving product data  Ikohs Netbot S18 Robot Vacuum Cleaner 4 on 1 with Mapping and App Alexa, Google  
Retrieving product data  Ecovacs Deebot N79S - Robot Vacuum Cleaner Navigation Random, Control App Alexa  
Retrieving product data  Ecovacs Deebot Slim2 Robot Vacuum Cleaner 3 on 1 Barre and Prune Steam Mop, App,  
Retrieving product data  Rozi Robot Vacuum Cleaner Super Silent 1600Pa and Floor Cleaner， Dual  
Retrieving product data  Cecotec Robot Vacuum Cleaner Conga Series 3290 Absolute 4 on 1 Technology Laser  
Retrieving product data  Cecotec Conga Series 1190 Robot Va